# Lecture 4. Process Inputs: Chain of Thought Reasoning

In [ ]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

#### 4.1. Introduction

In tasks to process inputs, i.e. the tasks that take the input and generate a  
useful output, often through a series of steps. It is sometimes important for the  
model to reason in detail about a problem before answering a specific question.  
Sometimes a model might make reasoning errors by rushing to an incorrect conclusion,  
so we can reframe the query to request a series of relevant reasoning steps before  
the model provides a final answer, so that it can think longer and more methodically  
about the problem. And in general, we call this strategy of asking the model to  
reason about a problem in steps, **Chain of Thought Reasoning**.

For some applications, the reasoning process that a model uses to arrive at a final  
answer would be inappropriate to share with the user. **Inner monologue** is a  
tactic that can be used to mitigate this. The idea of inner monologue is to instruct  
the model to put parts of the output that are meant to be hidden from the user into a  
structured format that makes passing them easy. Then, before presenting the output to  
the user, the output is passed and only part of the output is made visible. 

#### 4.2. Example Prompts where Chain-of-Thought Reasoning is used

##### Example 4.2.1

In [ ]:
delimiter = "####"

system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.

All available products:

1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [ ]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
]

In [ ]:
response = get_completion_from_messages(messages)

In [ ]:
# print(response)

# Step 1:#### The user is asking about the price difference between the
# BlueWave Chromebook and the TechPro Desktop.

# Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are available products.

# Step 3:#### The user assumes that the BlueWave Chromebook is more expensive than the TechPro Desktop.

# Step 4:#### Based on the product information, the TechPro Desktop is priced at $999.99, 
# and the BlueWave Chromebook is priced at $249.99. Therefore, the TechPro Desktop is actually 
# more expensive than the BlueWave Chromebook.

# Response to user:#### The BlueWave Chromebook is actually less expensive than the 
#TechPro Desktop. The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99.

##### Example 4.2.2

In [ ]:
ser_message = f"""
Do yo sell tvs"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{ser_message}{delimiter}"},  
]

In [ ]:
response = get_completion_from_messages(messages)

In [ ]:
# print(response)

# Step 1:#### The user is asking if the store sells TVs, which is a question about a specific product category.

# Step 2:#### TVs are not included in the list of available products. The store only sells computers and laptops.

# Response to user:#### I'm sorry, but we currently do not sell TVs. Our store specializes in computers  
# and laptops. If you have any questions or need assistance with our available products, feel free to ask.

#### 4.3. Example Prompts where where inner monologue is used to hide Chain-of-Thought Reasoning

In [ ]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."

In [ ]:
# print(final_response)

# I'm sorry, but we currently do not sell TVs. Our store specializes in computers and laptops.  
# If you have any questions or need assistance with our available products, feel free to ask.